<a href="https://colab.research.google.com/github/LucaChuang/TOTO-product-scraping/blob/main/toto_product_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
import csv
import io
warnings.filterwarnings('ignore')

In [3]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
# res = requests.get('https://www.twtoto.com.tw/product-single.aspx?cid=11&mid=4&type=1&id=17&page=1', headers = headers)
# res.encoding = 'utf-8'
# res.text
# bs = BeautifulSoup(res.text,'html.parser')

In [4]:
def getData(url):
    request_url= str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    bs=BeautifulSoup(res.text,'html.parser')

    if res.status_code == 200:
        bs=BeautifulSoup(res.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        model='NULL'
        prodtype='NULL'
        price='NULL'


        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        model = bs.find('div',{'class':'feature'}).findNext('h2').text
        prodtype = bs.find('h3').text
        price = bs.find('span',{'class':'price'}).text
        price = price[4:]
                
        return model, prodtype, price
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404

In [5]:
def getnote(url):
    request_url= str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    bs=BeautifulSoup(res.text,'html.parser')

    if res.status_code == 200:
        bs=BeautifulSoup(res.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        notes='NULL'

        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        notes = bs.find('li', {'class':'full'}).findNext('span').text
                
        return notes
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404

In [9]:
def main(outputfile, id, totalpages):

    
   
   with io.open(outputfile, "w", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["型號", "品名", "定價", "備註", "網址"])
        totalpages = totalpages
        print('Total pages: ', totalpages)

        for i in range(1, totalpages+1):
            request_url = "https://www.twtoto.com.tw/product-single.aspx?cid=11&mid=4&type=1&id="+str(id)+"&page="+str(i)
            res=requests.get(request_url, headers = headers)
            bs=BeautifulSoup(res.text,'html.parser')
            link_list = bs.find('div', id="product-list").findAll('a')

            url_list= [] #存放網址list
            for link in link_list:
              url = link.get('href')
              url_list.append(url)
              time.sleep(3)
            #time.sleep(random.randint(5,10))

            # ------------- write into csv ------------- #
            for url in url_list:
                model, prodtype, price = getData(url)
                try:
                  notes = getnote(url)
                except:
                  notes = "Null"
                writer.writerow([model, prodtype, price, notes, str(url)])
            # ------------------------------------------ #
            print(i/totalpages*100, '%') # print out 完成 %數

In [8]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    # 
    # link:
    id = 16           # 設定產品類別 (臉盆:16,淋浴:17,浴缸:18)
    outputfile = '/content/drive/MyDrive/output'+str(id)+'.csv' #設定存放位置與檔名
    totalpages = 5    # 設定產品頁數 (臉盆:5,淋浴:3,浴缸:2)
    # ---------------------------------------- #

    main(outputfile, id, totalpages)                                       #匯出csv檔
    read_file = pd.read_csv(outputfile)
    read_file.to_excel ('/content/drive/MyDrive/output'+str(id)+'.xlsx', index = None, header=True) #匯出excel檔
    print('\nfinish!')

Total pages:  5
20.0 %
40.0 %
60.0 %
80.0 %
100.0 %

finish!
